In [1]:
## Loading data

import pandas as pd
from datetime import datetime, timedelta

PATIENTS_FILE = "files/pd.trials.patient.csv"
TREATMENTS_FILE = "files/pd.trials.treatment.csv"
PAT_TREA_FILE = "files/pd.trials.patient_treatment.csv"
RESULTS_FILE = "files/pd.trials.lab_result.csv"

dfp = pd.read_csv(PATIENTS_FILE)
dft = pd.read_csv(TREATMENTS_FILE)
dfpt = pd.read_csv(PAT_TREA_FILE)
dfout = pd.read_csv(RESULTS_FILE)

# set dates with the appropriate type
dfp['EnrollDate'] = pd.to_datetime(dfp['EnrollDate'])
dfpt['StartDate'] = pd.to_datetime(dfpt['StartDate'])
dfout['VisitDate'] = pd.to_datetime(dfout['VisitDate'])


In [ ]:
## Queries

In [3]:
### Q. List the last 10 patients who enrolled in the trial program

last_10_patients = (
    dfp
    .sort_values('EnrollDate', ascending=False)  # Sort by date, newest first
    .head(10)                                     # Take first 10 rows
)

# or

last_10_patients = dfp.nlargest(10, 'EnrollDate')

last_10_patients

,PID,Age,Sex,SiteID,EnrollDate
23,124,55,M,LON-05,2025-05-31
49,150,62,F,LON-05,2025-05-28
17,118,36,F,LON-05,2025-05-24
46,147,60,M,LON-05,2025-05-22
7,108,53,M,NY-01,2025-05-20
19,120,78,F,LON-05,2025-05-20
42,143,38,M,NY-01,2025-05-18
15,116,37,F,TX-05,2025-05-10
41,142,52,M,NY-01,2025-04-28
22,123,36,F,TX-05,2025-04-25


In [4]:
### Q. Show all the trials started in March, reporting ID and starting date in a sorted way with respect to starting date.

dfpt[dfpt['StartDate'].dt.month == 3].sort_values('StartDate', ascending=True).reset_index(drop=True)[['TrtID','StartDate']]


,TrtID,StartDate
0,T01,2025-03-02
1,T01,2025-03-03
2,T01,2025-03-03
3,T01,2025-03-03
4,T03,2025-03-07
5,T02,2025-03-07
6,T02,2025-03-15
7,T03,2025-03-17
8,T01,2025-03-19
9,T01,2025-03-22


In [5]:
### Q. Show the complete list of all enrolled patients, showing their ID, sex, and the name of the drug they were assigned to.
'''
Expected outcome:

PID Sex      DrugName
101   M     Metabolix
102   F  StandardCare
103   F       Placebo
104   M     Metabolix
'''

# let us combine patients and their trial information
dfptreat = pd.merge(dfp, dfpt, on='PID')
# adding also the information on the treatment
dffull = pd.merge(dfptreat, dft, on='TrtID')

result = dffull[['PID', 'Sex', 'DrugName']]

In [6]:
### Q. Identify all patients over the age of 60 who were assigned to the _Metabolix_ protocol, reporting their IDs and ages.

dffull[(dffull.Age > 60) & (dffull.DrugName == 'Metabolix')][['PID','Age']]

# This solution requires two steps:
# 1. Row filtering: a temporary intermediate copy of the DataFrame containing all the columns is created for the matching rows
# 2. Column selection: the temporary DataFrame is sliced to keep only the two columns

# Alternative solution:

dffull.loc[
    (dffull.Age > 60) & (dffull.DrugName == 'Metabolix'), 
    ['PID', 'Age']
]

# Single Access: it only extracts the specific data points (intersection of those rows and those columns).

,PID,Age
10,113,70
11,114,74
12,114,74
18,120,78
20,122,72
21,122,72
26,128,78
28,130,61
29,130,61
30,130,61


In [7]:
### Q. Check the prescriptions at the New York (_NY-01_) site: list every patient in that site showing which drug they are on and the dosage they were prescribed.

dffull.loc[
    (dffull.SiteID == 'NY-01'), 
    ['PID', 'DrugName','Dosage_mg']
]


,PID,DrugName,Dosage_mg
1,102,StandardCare,20
4,106,Metabolix,20
6,108,Metabolix,20
8,110,StandardCare,20
9,110,StandardCare,10
11,114,Metabolix,50
12,114,Metabolix,40
13,114,Placebo,0
19,121,StandardCare,20
24,125,StandardCare,10


In [9]:
### Q. Verify the randomization for the _Placebo_ group: retrieve the number of females and males in such a group (create two variables numPlaM and numPlaF and print them).

numPlaMF = dffull.loc[
    (dffull.DrugName == 'Placebo'), 
    ['Sex']
].value_counts()

# it returns an array with names
'''
Sex
M      8
F      2
Name: count, dtype: int64
'''

numPlaM = numPlaMF['M']
numPlaF = numPlaMF['F']

# Alternative solution

# Get gender distribution in Placebo group
numPlaMF = (
    dffull[dffull['DrugName'] == 'Placebo']         # Filter for Placebo drug
    ['Sex']                                         # Select only Sex column
    .value_counts()                                 # Count occurrences of each sex
    .sort_index()                                   # Sort by Sex (F, M)
)

numPlaM, numPlaF

(8, 3)

In [11]:
### Q. What is the average _Systolic Blood Pressure_ (TestName is _Systolic_BP_) across the entire study population, regardless of what drug they are on?

# separate mask to filter rows
bpmask = (dfout['TestName'] == 'Systolic_BP')

# creating an intermediate dataFrame
dfbp = dfout[bpmask]
avgBloodPressure = dfbp['Value'].mean()

# or 

dfbp = dfout.loc[bpmask]
avgBloodPressure = dfbp['Value'].mean()

# we do not really need the intermediate dataframe
# without creating an intermediate dataframe
avgBloodPressure = dfout.loc[bpmask,'Value'].mean()

avgBloodPressure

137.06075949367087

In [12]:
### Q. Compare the three study arms. What is the average _HbA1c_ level for the _Metabolix_ group, compared to _StandardCare_ and the _Placebo_ group
'''
Method:

1. **Split** data into groups (by Drug).
2. **Apply** a function (Mean).
3. **Combine** the results into a new table.
'''

# 0. Combine treatments and lab_results for the same patient
dftpout = pd.merge(dfpt, dfout, on='PID')
dftpout = pd.merge(dftpout, dft, on='TrtID')


# 1. Filter: Select only HbA1c records
df_hba1c = dftpout.loc[dftpout.TestName == 'HbA1c']

# 2. GroupBy: Calculate the mean value per Drug
# syntax: df.groupby('Grouping_Column')['Math_Column'].mean()
dfres = df_hba1c.groupby('DrugName')['Value'].mean().reset_index(name='avgHbA1c')

dfres

,DrugName,avgHbA1c
0,Metabolix,6.765657
1,Placebo,7.317143
2,StandardCare,7.289474


In [13]:
### Q. Which patient has the most data points? Find the Patient ID associated with the _highest number of individual lab records_.

# 1. Group by PID and count the LabIDs (or any column)
# This creates a Series where index=PID and value=Count
counts = dftpout.groupby('PID')['LabID'].count().reset_index(name='NumberRecords')

# 2. Sort descending to get the highest at the top
top_row = counts.sort_values(by='NumberRecords', ascending=False).head(1)

# 3. Show the top patient
print(top_row['PID'].iloc[0])

114


In [15]:
### Q. What is the single _highest LDL value_ recorded for any patient on the _Experimental_ protocol?

# Define the protocol you are looking for (e.g., 'Experimental', 'ActiveControl', or 'PlaceboControl')
protocol = 'Experimental'

# Filter dftpout for:
# 1. The specific TestName ('LDL')
# 2. The specific StudyArm
highest_ldl = dftpout[
    (dftpout['TestName'] == 'LDL') & 
    (dftpout['StudyArm'] == protocol)
]['Value'].max()

highest_ldl

197.0

In [16]:
### Q. List all lab results (ID, Test, Value, Date) that were taken _before_ the patient's medication Start Date, this sets the baseline data

'''
1. We need to bring the StartDate (from the treatment table) onto the results table so that every lab row has a "reference date" to compare against -> merge
2. Keep rows where VisitDate < StartDate -> filter
'''

mask = (dftpout['VisitDate'] < dftpout['StartDate'])

baseline_data = dftpout.loc[
    mask, 
    ['LabID', 'TestName', 'Value', 'VisitDate', 'StartDate']
]

print(baseline_data.head())


    LabID     TestName  Value  VisitDate  StartDate
0       1        HbA1c    6.4 2025-01-31 2025-02-02
1       2  Systolic_BP  151.0 2025-01-31 2025-02-02
2       3          LDL  133.0 2025-01-31 2025-02-02
3       4    Weight_kg  105.5 2025-01-31 2025-02-02
12     13        HbA1c    5.8 2025-01-11 2025-01-13


In [18]:
### Q. Calculate the average number of days between a patient's _Enrollment Date_ and their treatment _Start Date_, by _Site ID_.
'''
1. Join patients (Enroll Date & Site) with patient_treatment (Start Date) -> merge (`dffull`)
2. Create a new column called deltaTime by subtracting EnrollDate from StartDate and extract the number of days.
3. Group by SiteID and average the numbers -> `group by` and `average`
'''

dffull['deltaTime'] = dffull['StartDate'] - dffull['EnrollDate']
dffull['deltaDays'] = dffull['deltaTime'].dt.days

# also in a single step
dffull['deltaDays'] = (dffull['StartDate'] - dffull['EnrollDate']).dt.days
# group and average
dfDeltaDays = dffull.groupby('SiteID')['deltaDays'].mean().reset_index(name='avgDeltaDays')

dfDeltaDays

,SiteID,avgDeltaDays
0,LON-05,8.000000
1,NY-01,10.823529
2,TX-05,11.500000


In [19]:
### Q. Are there any 'Waiting Patients'? List the IDs of any patients who are enrolled in the system but have no associated treatment so far

# Looking for patients that do not have an entry in `patient_treatments`

# the important part is **left**
dftmp = pd.merge(dfp, dfpt, on='PID', how='left')

dftmp.loc[dftmp.TrtID.isna(),'PID']


4     105
11    111
12    112
21    119
29    126
38    133
45    140
52    147
53    148
60    154
Name: PID, dtype: int64

In [20]:
### Q. Are there any 'Ghost Patients'? List the IDs of any patients who are enrolled in the system but have _zero_ lab results recorded in the database so far

# Similar query

dftmp = pd.merge(dfp, dfout, on='PID', how='left')
dftmp.loc[dftmp.LabID.isna(),'PID']

## alternative using isin

# take all PIDs of patients whose PID is not in results
dfp[~dfp['PID'].isin(dfout['PID'])]['PID']


4     105
10    111
11    112
18    119
25    126
32    133
39    140
46    147
47    148
53    154
Name: PID, dtype: int64

In [22]:
### Q. Create a summary table with _one row per patient_, showing their average monitored values in separate columns

# Reshaping

# Create the Pivot Table
# We use pivot_table() instead of pivot() because we have multiple dates.
# We need to AGGREGATE (mean) the values from different dates into one number.
dfout_sum = dfout.pivot_table(
    index='PID',           # What defines the rows?
    columns='TestName',    # What defines the new column headers?
    values='Value',        # What numbers go inside?
    aggfunc='mean'         # How do we combine multiple visits? (Average)
).reset_index()

dfout_sum

TestName,PID,HbA1c,LDL,Systolic_BP,Weight_kg
0,101,6.466667,119.000000,159.333333,98.833333
1,102,5.866667,104.333333,148.666667,90.600000
2,103,6.433333,126.666667,148.000000,89.400000
3,104,6.533333,123.666667,132.333333,102.366667
4,106,5.766667,123.333333,142.000000,90.166667
5,107,6.966667,133.666667,155.000000,77.066667
6,108,7.900000,122.666667,129.333333,96.300000
7,109,6.066667,134.000000,127.000000,74.600000
8,110,8.300000,107.600000,152.600000,91.800000
9,113,6.633333,114.000000,156.333333,81.200000


In [24]:
### Q. When looking at _Metabolix_ users specifically, is there a difference in average _Weight_ between Male and Female patients in that group?

# `Metabolix` is the `DrugName` for the patients we are interested in. For those data, we need to compute the average `Weight_kg` for the different groups based on `Sex`.

# Merge dftout with dfp to have also the sex information
dfall = pd.merge(dftpout, dfp, on='PID')

# Filter the data for:
#  - DrugName: 'Metabolix'
#  - TestName: 'Weight_kg'
metabolix_weight = dfall[
    (dfall['DrugName'] == 'Metabolix') & 
    (dfall['TestName'] == 'Weight_kg')
]

# Group by Sex and calculate the average (mean) Value
avgWeight = metabolix_weight.groupby('Sex')['Value'].mean()

avgWeight

Sex
F    84.208955
M    93.034375
Name: Value, dtype: float64

In [26]:
### Q. List patient IDs for anyone who has ever recorded a _Systolic BP > 160_ OR an _LDL > 150_ at any point in the study

# selection based on 
# Condition 1: Test is Systolic_BP AND Value is > 160
# OR (|)
# Condition 2: Test is LDL AND Value is > 150
highRisk_filter = (
    ((dftpout['TestName'] == 'Systolic_BP') & (dftpout['Value'] > 160)) | 
    ((dftpout['TestName'] == 'LDL') & (dftpout['Value'] > 150))
)

# Apply the filter to get the rows, select the PID column, 
# and get unique values (so a patient isn't listed twice)
fdrisk = list(dftpout.loc[highRisk_filter, 'PID'].unique())
fdrisk

[101,
 103,
 107,
 109,
 110,
 113,
 114,
 115,
 118,
 121,
 124,
 129,
 131,
 136,
 137,
 144,
 146,
 150,
 151,
 155,
 159]

In [31]:
# 1. Calculate 'DaysOnTreatment' relative to the start date
dfall['VisitDate'] = pd.to_datetime(dfall['VisitDate'])
dfall['StartDate'] = pd.to_datetime(dfall['StartDate'])
dfall['DaysOnTreatment'] = (dfall['VisitDate'] - dfall['StartDate']).dt.days

# 2. Filter specifically for HbA1c
hba1c_df = dfall[dfall['TestName'] == 'HbA1c'].copy()

# 3. Identify Baseline measurements (approx Day 0, e.g., <= 5 days)
# We rename 'Value' to 'Baseline_Value' for clarity
baseline = hba1c_df[hba1c_df['DaysOnTreatment'] <= 5][['PID', 'Value']].rename(
    columns={'Value': 'Baseline_Value'}
)

# 4. Identify Day 60 measurements (we look for a window, e.g., 50 to 70 days)
# We rename 'Value' to 'Day60_Value'
day60 = hba1c_df[
    (hba1c_df['DaysOnTreatment'] >= 50) & 
    (hba1c_df['DaysOnTreatment'] <= 70)
][['PID', 'Value']].rename(columns={'Value': 'Day60_Value'})

# 5. Merge Baseline and Day60 data on PID
# An inner join ensures we only analyze patients who have BOTH measurements
analysis_df = pd.merge(baseline, day60, on='PID')

# 6. Calculate the Reduction (Start - End)
# Positive number = Improvement (HbA1c went down)
analysis_df['Reduction'] = analysis_df['Baseline_Value'] - analysis_df['Day60_Value']

# 7. Add DrugName information back to this table
# We get a mapping of PID -> DrugName from the original dataframe
patient_drugs = dfall[['PID', 'DrugName']].drop_duplicates()
analysis_df = pd.merge(analysis_df, patient_drugs, on='PID')

# 8. Calculate average reduction by Drug
average_reduction = analysis_df.groupby('DrugName')['Reduction'].mean()

print("Average HbA1c Reduction over 60 days (Higher is better):")
print(average_reduction.sort_values(ascending=False))

Average HbA1c Reduction over 60 days (Higher is better):
DrugName
Metabolix       1.340000
StandardCare    0.875000
Placebo         0.383333
Name: Reduction, dtype: float64
